#Clone repository containing test data



In [ ]:
%cd /content/
! git clone https://github.com/ToastyDom/DataChallengesSoSe22.git

/content
Cloning into 'DataChallengesSoSe22'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 30 (delta 5), reused 13 (delta 0), pack-reused 0
Unpacking objects: 100% (30/30), done.


# Install the latest master of Haystack


In [ ]:
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 4.7 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-mbnin5c2
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-mbnin5c2
  Resolved https://github.com/deepset-ai/haystack.git to commit 4e45062a00da18c7c039c24b1ea70641e79f82a8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.7/219.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.7 MB/s eta

## Run Elastic Server

In [ ]:
# Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

### Import relevant packages

In [ ]:
import json
from pprint import pprint
import logging

from haystack.document_stores import ElasticsearchDocumentStore
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader

# Important Imports
from haystack.pipelines import ExtractiveQAPipeline
from haystack.nodes import BM25Retriever
from haystack.pipelines import GenerativeQAPipeline
from haystack.nodes import RAGenerator
from haystack.nodes import BM25Retriever, EmbeddingRetriever, FARMReader
from haystack.pipelines import DocumentSearchPipeline
from haystack.utils import print_documents
from haystack.nodes import BM25Retriever
from haystack import Pipeline
from haystack.nodes import EmbeddingRetriever

### Taking the JSON-Data and putting it into the neccessary format

In [ ]:
file = open('DataChallengesSoSe22/data/db_bahn_tweets_2_years.json', 'r')
lines = file.readlines()

extracted_tweets = []

for line in lines:
  tweet = json.loads(line)
  extracted_tweets.append([
    tweet["username"],
    tweet["created_at"],
    tweet["tweet"],
    tweet["id"],
    tweet["link"]
  ])

In [ ]:
"""Testing if all the tweets are shown"""

print(len(extracted_tweets))

for tweet in extracted_tweets[0:10]:
  print(tweet)

71379
['db_bahn', '2020-12-31 21:00:37 UTC', 'Ihr Lieben, wir verabschieden uns nun für dieses Jahr von euch! 🎉 Rutscht oder rollt (ich rolle, weil vollgefuttert 🙈 😂) ins neue Jahr 2021! 🥳 Richtet den Blick voller Zuversicht nach vorne, fühlt euch von uns gedrückt und bleibt gesund. 🍀 🥂 /at', 1344750330848931846, 'https://twitter.com/DB_Bahn/status/1344750330848931846']
['db_bahn', '2020-12-31 20:46:37 UTC', '@atYildir Schon geschehen. /da', 1344746806383669268, 'https://twitter.com/DB_Bahn/status/1344746806383669268']
['db_bahn', '2020-12-31 20:40:59 UTC', '@atYildir Da die Strecke von der HLB betreiben wird, haben wir von den Ticketeinnahmen nichts. Dort verkehren aber auch Züge der Linie RB 41, die von DB Regio betrieben wird. Wann möchtest du von Frankfurt Hbf nach Marburg(Lahn) fahren? /da', 1344745390076260359, 'https://twitter.com/DB_Bahn/status/1344745390076260359']
['db_bahn', '2020-12-31 18:53:31 UTC', '@SebastianKueck Danke für die lieben Worte. Wir wünschen dir einen guten 

# Adding data to Haystack Pipeline
Reader: Close Anaylsis of documents, perform core task of question answering. Trained from the latest transformer

Retriever: Assists reader as filter for documents. Quickly identifies relevant parts

Haystack looks at documents stored in a class called "Document Store"

First we need to start an ElasticSearch (like a Bert-Model) server on local machine. Usually using docker but we can execute elastic search from source as well.

In [ ]:
# Connect to Elasticsearch

document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

INFO - haystack.telemetry -  Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


Now we have Elastic Search running and a yet-empty document_store. Now we need to fill our document store with our data!



In [ ]:
# Write function to put json data into document store format
"""
dicts = [
    {
        'content': DOCUMENT_TEXT_HERE,
        'meta': {'name': DOCUMENT_NAME, ...}
    }, ...
]

[author_name, tweet_date, tweet, id, url]
"""

def to_dstore(json_data):
  dicts = []
  for element in json_data:
    current_dict = {'content': element[2],
                    'meta': {'author': element[0], 'date': element[1], 'url': element[4]}}
    dicts.append(current_dict)

  return dicts

In [ ]:
docs = to_dstore(extracted_tweets)
document_store.write_documents(docs)

# Extractive Q&A Pipeline

In [ ]:
retriever = EmbeddingRetriever(
    document_store=document_store,
   embedding_model="clips/mfaq",
   model_format="sentence_transformers"
)
document_store.update_embeddings(retriever)

# Adding reader.
reader = FARMReader(model_name_or_path="deepset/gelectra-base-germanquad", use_gpu=True)

p_extractive = Pipeline()
p_extractive.add_node(component=retriever, name="Retriever", inputs=["Query"])
p_extractive.add_node(component=reader, name="Reader", inputs=["Retriever"])

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.nodes.retriever.dense -  Init retriever using embeddings of model clips/mfaq


Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

INFO - haystack.modeling.model.language_model -  Auto-detected model language: german
INFO - haystack.modeling.model.language_model -  Loaded 'deepset/gelectra-base-germanquad' (Electra model) from model hub.


Downloading:   0%|          | 0.00/358 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/234k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0  
INFO - haystack.modeling.infer -  /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / \ 


In [ ]:
# Now we can run it
res = p_extractive.run(
    query="Wiviel kostet das 9€ Ticket?", params={"Retriever": {"top_k": 30}, "Reader": {"top_k": 3}}
)
print_answers(res, details="maximum")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 55.24 Batches/s]
WARNING - root -  print_answers received details='maximum', which was not understood. 
WARNING - root -  Valid values are minimum, medium and 'all'. Using 'all'.



Query: Wiviel kostet das 9€ Ticket?
Answers:
[   <Answer {'answer': '46 Euro', 'type': 'extractive', 'score': 0.8925023674964905, 'context': 'en Verbundshop gegangen? Ich habe es gerade auf iOS getestet und kann dort ein 10er Ticket ab Korschenbroich Geltungsbereich B für 46 Euro buchen. /ne', 'offsets_in_document': [{'start': 193, 'end': 200}], 'offsets_in_context': [{'start': 131, 'end': 138}], 'document_id': 'b0d82054126a22c2bebcd194c9404fd9', 'meta': {'date': '2021-11-12 14:20:35 UTC', 'author': 'db_bahn', 'url': 'https://twitter.com/DB_Bahn/status/1459164221464272901'}}>,
    <Answer {'answer': '151,15 Euro', 'type': 'extractive', 'score': 0.7013881802558899, 'context': 'cht, wie du auf 194 Euro kommst, der Flexpreis mit BahnCard 25 kostet 151,15 Euro, Super Sparpreis Europa für morgen gibt es noch für 97,90 Euro. Hast', 'offsets_in_document': [{'start': 105, 'end': 116}], 'offsets_in_context': [{'start': 70, 'end': 81}], 'document_id': '62b648964e38953fcc3a9b812e20a0ea', 'meta':

# Frontend Interface

In [ ]:
# Disable info logs
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

def ask_chatbot(question):
  if question == "":
    return "", ""
  prediction = p_extractive.run(
    query=question,
    params={"Retriever": {"top_k": 30}, "Reader": {"top_k": 1}}
  )
  if prediction["answers"]:
    return prediction["answers"][0].answer, prediction["answers"][0].meta["url"]
  else:
    return "", ""

In [ ]:
from ipywidgets import *


style = HTML(value="""<style>
.app { 
  background-color: #eba8a2;
  padding: 50px 200px;
  max-width: 1000px;
}
.logo {
  background: url(https://raw.githubusercontent.com/ToastyDom/DataChallengesSoSe22/main/logo.png);
  height: 80px;
  background-position: center;
  background-repeat: no-repeat;
  background-size: contain;
}
.messages {
  height: 300px;
  margin: 40px 0;
  overflow-y: scroll;
  flex-direction: column-reverse;
}
.message {
  padding: 10px;
  margin: 5px 0px;
  background-color: #f5e1df;
  border-radius: 10px;
  width: 70%;
}
a {
  color: black;
  text-decoration: none;
}
.message.right {
  margin-left: 25%;
  border-top-right-radius: 0px;
}
.message.left {
  border-top-left-radius: 0px;
}
.input {
  width: 75%;
}
.button {
  width: 25%;
}
</style>""")

input = Text(placeholder="Frage eingeben")
button = Button(description="Fragen")
messages = VBox([
  HTML(value='<div class="message left">Hi, wie kann ich dir helfen?</div>'),
])
messages.add_class('messages')
app = VBox([
  HTML(value='<div class="logo"></div>'),
  messages,
  HBox([input, button]),
])
app.add_class('app')
input.add_class('input')
button.add_class('button')

def on_send_message(b):
  question = input.value
  answer, url = ask_chatbot(question)
  messages.children = (
    HTML(value=f'<a href="{url}" target="_blank"><div class="message left">{answer}</div></a>'),
    HTML(value=f'<div class="message right">{question}</div>'),
    *messages.children,
  )

button.on_click(on_send_message)
input.on_submit(on_send_message)


display(style, app)

HTML(value='<style>\n.app { \n  background-color: #eba8a2;\n  padding: 50px 200px;\n  max-width: 1000px;\n}\n.…

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 55.33 Batches/s]
